In [21]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from util_checklist import sample_testcases, get_ingredient_set
import json
# https://github.com/marcotcr/checklist/blob/master/notebooks/tutorials/1.%20Generating%20data.ipynb 

In [2]:
ingredients_tups = get_ingredient_set('recipe_ingredient_dataset/train.json', 'recipe_ingredient_dataset/test.json')

49718 4971.8


In [3]:
editor = Editor()

In [4]:
print(list(editor.lexicons.keys()))

['sentiment', 'male', 'female', 'first_name', 'first_pronoun', 'last_name', 'country', 'nationality', 'city', 'religion', 'religion_adj', 'sexual_adj', 'country_city', 'male_from', 'female_from', 'last_from']


In [5]:
ing_1_sentences = ['I would like a dish with {ing}', 
                   'Show me {ing} recipes', 
                   'Can I see recipes with {ing}',
                   'I want {ing} in my recipe']
# TODO: come up with a way to generate ing indices.
se_tups = {}
for i in range(len(ing_1_sentences)):
    sentence = ing_1_sentences[i]
    start_indx = sentence.index('{ing}')
    end_indx = start_indx + 5
    end_indx_frm_end = len(sentence) - end_indx
    se_tups[i] = (start_indx, end_indx_frm_end)

def add_type_info(data):
    return [str(i) + '\t' + data[i] for i in range(len(data))]

In [13]:
ingredients = [x[0] for x in ingredients_tups]

In [14]:
# single ingredient
ret = editor.template(add_type_info(ing_1_sentences), 
                      ing=ingredients, 
                      unroll=True,
                      remove_duplicates=True)

In [15]:
small_dataset = sample_testcases(ret.data, 200)

In [16]:
small_dataset

array(['2\tCan I see recipes with tomato paste',
       '0\tI would like a dish with all-purpose flour',
       '0\tI would like a dish with red bell pepper',
       '3\tI want baking soda in my recipe',
       '1\tShow me fresh parsley recipes',
       '0\tI would like a dish with hot sauce',
       '1\tShow me garlic powder recipes',
       '1\tShow me kosher salt recipes',
       '3\tI want black pepper in my recipe',
       '3\tI want celery ribs in my recipe',
       '3\tI want chicken stock in my recipe',
       '0\tI would like a dish with boneless skinless chicken breasts',
       '1\tShow me medium shrimp recipes',
       '2\tCan I see recipes with cherry tomatoes',
       '3\tI want white onion in my recipe',
       '3\tI want large eggs in my recipe',
       '3\tI want ground pork in my recipe',
       '0\tI would like a dish with oregano',
       '0\tI would like a dish with worcestershire sauce',
       '0\tI would like a dish with dried basil',
       '0\tI would like a d

In [17]:
def convert_checklist_ds(ck_dataset, se_tups):
    label_name = "INGREDIENT"
    converted_ds = []
    for data in ck_dataset:
        sen_type, text = data.split('\t')
        start,end = se_tups[int(sen_type)]
        label_start = start
        label_end = len(text) - end
        annotated = [text]
        annotated.append({'entities': [[label_start, label_end, label_name]]})
        converted_ds.append(annotated)
    return converted_ds

In [19]:
converted_ds_list = convert_checklist_ds(small_dataset, se_tups)

In [27]:
with open('checklist.json', 'w') as fobj:
    json.dump(converted_ds_list, fobj)